## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf
import numpy as np

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
tf.enable_eager_execution()

### Collect Data

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/gdrive/My Drive/prices.csv')

### Check all columns in the dataset

In [10]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [11]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(['date','symbol'], axis = 1)

In [13]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [14]:
data.shape

(851264, 5)

In [15]:
data = data.iloc[:1000,:]
data.shape

(1000, 5)

In [16]:
data['volume'] = data['volume']/1000000
data['volume'].describe()

count    1000.000000
mean        5.314222
std        14.465650
min         0.010000
25%         0.821625
50%         2.064450
75%         4.620475
max       215.620200
Name: volume, dtype: float64

In [0]:
X = data.drop('volume', axis = 1)
Y = data['volume']

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

#### Convert Training and Test Data to numpy float32 arrays


In [19]:
np.array(train_x).dtype

dtype('float64')

In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

In [21]:
np.array(train_x).dtype

dtype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer() #same as Z-score
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

In [23]:
train_y.shape

(700,)

In [0]:
train_y=train_y.reshape(-1,1)

In [25]:
train_y.shape

(700, 1)

In [26]:
train_x.shape

(700, 4)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias of (Input-Hidden Layer)
w1 = tf.random_normal(shape=(4,4))
b1 = tf.zeros(shape=(4))

In [0]:
#We are initializing weights and Bias of (Hidden-Output)
w2 = tf.random_normal(shape=(4,1))
b2 = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w1, b1,w2,b2):
    
    xw_matmul = tf.matmul(x, w1)
    net1 = tf.add(xw_matmul, b1)
    y=tf.sigmoid(net1)
    net2=tf.matmul(y, w2)+b2
    out=tf.sigmoid(net2)
    return net2,out

## net2 for regression and out for classification

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(train_x, train_y, w1, b1,w2,b2, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w1,b1,w2,b2]) #which values have to be updated/keep track of/model parameters
        
        net2,current_prediction = prediction(train_x, w1, b1,w2,b2)
        current_loss =loss(net2, train_y)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(current_loss,[w1, b1,w2,b2])
    
    #Update Weights at output layer
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1
    
    return w1, b1,w2,b2, current_loss

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [52]:
for i in range(100):
    
    w1, b1,w2,b2, current_loss = train(train_x, train_y, w1, b1,w2,b2)
    print("Loss at step {:d}: {:.3f}".format(i, current_loss))

Loss at step 0: 240.426
Loss at step 1: 237.834
Loss at step 2: 235.530
Loss at step 3: 233.484
Loss at step 4: 231.671
Loss at step 5: 230.067
Loss at step 6: 228.650
Loss at step 7: 227.401
Loss at step 8: 226.301
Loss at step 9: 225.335
Loss at step 10: 224.487
Loss at step 11: 223.743
Loss at step 12: 223.092
Loss at step 13: 222.523
Loss at step 14: 222.025
Loss at step 15: 221.589
Loss at step 16: 221.210
Loss at step 17: 220.878
Loss at step 18: 220.588
Loss at step 19: 220.336
Loss at step 20: 220.116
Loss at step 21: 219.924
Loss at step 22: 219.757
Loss at step 23: 219.611
Loss at step 24: 219.484
Loss at step 25: 219.373
Loss at step 26: 219.277
Loss at step 27: 219.193
Loss at step 28: 219.119
Loss at step 29: 219.056
Loss at step 30: 219.000
Loss at step 31: 218.952
Loss at step 32: 218.910
Loss at step 33: 218.873
Loss at step 34: 218.841
Loss at step 35: 218.813
Loss at step 36: 218.789
Loss at step 37: 218.768
Loss at step 38: 218.749
Loss at step 39: 218.733
Loss at st

In [53]:
j = 0
for i in range(100):
    j = j+1
    w1, b1,w2,b2, current_loss = train(train_x, train_y, w1, b1,w2,b2)
    if(j ==5):
      print("Loss at step {:d}: {:.3f}".format(i, current_loss))
      j = 0

Loss at step 4: 218.624
Loss at step 9: 218.624
Loss at step 14: 218.624
Loss at step 19: 218.624
Loss at step 24: 218.624
Loss at step 29: 218.624
Loss at step 34: 218.624
Loss at step 39: 218.624
Loss at step 44: 218.624
Loss at step 49: 218.624
Loss at step 54: 218.624
Loss at step 59: 218.624
Loss at step 64: 218.624
Loss at step 69: 218.624
Loss at step 74: 218.624
Loss at step 79: 218.624
Loss at step 84: 218.624
Loss at step 89: 218.624
Loss at step 94: 218.624
Loss at step 99: 218.624


### Get the shapes and values of W and b

In [54]:
w1.shape

TensorShape([Dimension(4), Dimension(4)])

In [55]:
w2.shape

TensorShape([Dimension(4), Dimension(1)])

In [56]:
b1.shape

TensorShape([Dimension(4)])

In [57]:
b2.shape

TensorShape([Dimension(1)])

### Model Prediction on 1st Examples in Test Dataset

In [58]:
y_pred,_=prediction(test_x,w1,b1,w2,b2)
y_pred

<tf.Tensor: id=21669, shape=(300, 1), dtype=float32, numpy=
array([[5.6505485],
       [5.6470766],
       [5.646221 ],
       [5.6585016],
       [5.647886 ],
       [5.6536746],
       [5.6453934],
       [5.6530647],
       [5.6459804],
       [5.6460166],
       [5.646792 ],
       [5.6514344],
       [5.656713 ],
       [5.6501236],
       [5.6477604],
       [5.653584 ],
       [5.645592 ],
       [5.647278 ],
       [5.651734 ],
       [5.6554346],
       [5.6457977],
       [5.648939 ],
       [5.6448193],
       [5.6547084],
       [5.6465425],
       [5.6528373],
       [5.6493077],
       [5.645209 ],
       [5.6478553],
       [5.6512504],
       [5.644221 ],
       [5.645487 ],
       [5.647436 ],
       [5.648266 ],
       [5.6528187],
       [5.647716 ],
       [5.652009 ],
       [5.6498265],
       [5.645916 ],
       [5.6541834],
       [5.644685 ],
       [5.653973 ],
       [5.6469655],
       [5.6490035],
       [5.6473293],
       [5.65677  ],
       [5.646687 ],


In [59]:
test_y - y_pred

<tf.Tensor: id=21672, shape=(300, 300), dtype=float32, numpy=
array([[-4.6789484, -3.6104484, -2.1939485, ..., -2.7247484, -4.2318487,
        -5.1679482],
       [-4.6754766, -3.6069765, -2.1904767, ..., -2.7212765, -4.2283764,
        -5.1644764],
       [-4.674621 , -3.606121 , -2.1896212, ..., -2.720421 , -4.227521 ,
        -5.163621 ],
       ...,
       [-4.6786876, -3.6101875, -2.1936877, ..., -2.7244875, -4.2315874,
        -5.1676874],
       [-4.6778703, -3.6093702, -2.1928704, ..., -2.7236702, -4.23077  ,
        -5.16687  ],
       [-4.6770954, -3.6085954, -2.1920955, ..., -2.7228954, -4.2299957,
        -5.1660953]], dtype=float32)>

In [62]:
from sklearn import metrics
cm=metrics.r2_score(test_y,y_pred)
print(cm)

-0.006542277548792885


In [67]:
loss(y_pred, test_y)

<tf.Tensor: id=21677, shape=(), dtype=float32, numpy=187.02565>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
data_iris = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [64]:
data_iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [0]:
X = data_iris.drop('Species', axis = 1)
Y = data_iris['Species']

In [69]:
X.shape

(150, 5)

In [71]:
X = X.drop('Id', axis = 1)
X.shape

(150, 4)

In [70]:
Y.shape

(150,)

In [77]:
Y=np.array(Y).reshape(-1,1)
Y = pd.DataFrame(Y)
Y.shape

(150, 1)

In [0]:
Y['species'] = Y

In [0]:
Y = Y.drop(0, axis = 1)

In [82]:
Y.head()

,species
0,Iris-setosa
1,Iris-setosa
2,Iris-setosa
3,Iris-setosa
4,Iris-setosa


In [85]:
Y =pd.get_dummies(Y)
Y.head()

,species_Iris-setosa,species_Iris-versicolor,species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [0]:
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
from keras.models import Sequential
from keras.layers import Dense


In [0]:
tf.compat.v1.disable_eager_execution()
model = Sequential()
model.add(Dense(4, input_dim = 4, kernel_initializer='normal', activation='sigmoid'))
model.add(Dense(3, activation='softmax'))    
    # Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics = ['accuracy'])


### Model Training 

In [137]:

output = tf.Session().run(model.fit(train_x, train_y,validation_data = (test_x,test_y), epochs = 50))
print(output)

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 0s 2ms/step - loss: 1.1269 - acc: 0.3333 - val_loss: 1.0992 - val_acc: 0.3333
Epoch 2/50
105/105 [==============================] - 0s 88us/step - loss: 1.1201 - acc: 0.3333 - val_loss: 1.0959 - val_acc: 0.3333
Epoch 3/50
105/105 [==============================] - 0s 81us/step - loss: 1.1156 - acc: 0.3333 - val_loss: 1.0926 - val_acc: 0.3333
Epoch 4/50
105/105 [==============================] - 0s 77us/step - loss: 1.1107 - acc: 0.3333 - val_loss: 1.0892 - val_acc: 0.3333
Epoch 5/50
105/105 [==============================] - 0s 91us/step - loss: 1.1070 - acc: 0.3333 - val_loss: 1.0863 - val_acc: 0.3333
Epoch 6/50
105/105 [==============================] - 0s 87us/step - loss: 1.1037 - acc: 0.3333 - val_loss: 1.0828 - val_acc: 0.3333
Epoch 7/50
105/105 [==============================] - 0s 86us/step - loss: 1.0993 - acc: 0.3333 - val_loss: 1.0809 - val_acc: 0.3333
Epoch 8/50
105/105 [=====

TypeError: ignored

### Model Prediction

In [138]:

y_pred = model.predict(test_x)
y_pred

array([[0.40436143, 0.32286045, 0.27277812],
       [0.39641473, 0.32811564, 0.27546966],
       [0.28820595, 0.39492518, 0.3168689 ],
       [0.39785194, 0.32668558, 0.27546242],
       [0.39225462, 0.32925293, 0.2784925 ],
       [0.28715953, 0.3885046 , 0.3243359 ],
       [0.39081445, 0.3302695 , 0.27891606],
       [0.27877724, 0.39751732, 0.32370538],
       [0.28319937, 0.395065  , 0.3217357 ],
       [0.39500636, 0.32856244, 0.27643123],
       [0.40846714, 0.31737283, 0.2741601 ],
       [0.40432823, 0.32389435, 0.27177748],
       [0.4058047 , 0.3209684 , 0.27322695],
       [0.40458855, 0.3190801 , 0.2763313 ],
       [0.31435382, 0.3739397 , 0.31170648],
       [0.30362704, 0.3803893 , 0.31598368],
       [0.38252208, 0.3386474 , 0.27883053],
       [0.31456533, 0.37587845, 0.30955622],
       [0.2858651 , 0.3912318 , 0.32290304],
       [0.31156507, 0.3795505 , 0.3088844 ],
       [0.29833403, 0.38283694, 0.31882906],
       [0.3124679 , 0.37459227, 0.31293985],
       [0.

In [0]:

y_test_class = np.argmax(test_y,axis = 1)
y_pred_class = np.argmax(y_pred, axis = 1)

In [150]:
y_pred_class

array([0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0])

In [148]:
from sklearn.metrics import classification_report, confusion_matrix
sess.run(print(classification_report(y_test_class,y_pred_class)))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       0.54      1.00      0.70        15
           2       0.00      0.00      0.00        13

    accuracy                           0.71        45
   macro avg       0.51      0.67      0.57        45
weighted avg       0.56      0.71      0.61        45



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


TypeError: ignored

In [149]:
sess.run(print(confusion_matrix(y_test_class,y_pred_class)))

[[17  0  0]
 [ 0 15  0]
 [ 0 13  0]]


TypeError: ignored

### Save the Model

In [0]:
model.save('iris_model.py')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?